<a href="https://colab.research.google.com/github/marekpiotradamczyk/ml_uwr_22/blob/main/kmeans_deep_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How it differs from the default solution? 

Here we are using a neural network instead of a logreg

# Load data & default imports

In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import scipy.stats as sstats
import multiprocessing as mp
from sklearn import datasets
import sklearn.linear_model
from tqdm.auto import tqdm
from matplotlib import animation, pyplot, rc
import matplotlib.pyplot as plt
import httpimport
from os.path import join
import os.path
from PIL import Image

from sklearn.cluster import KMeans, MiniBatchKMeans
import numpy as np

from sklearn.feature_extraction import image
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.linear_model import LogisticRegression

from FeaturesExtractor import FeaturesExtractor

import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten

!pip install -q gdown httpimport
![ -e cifar.npz ] || gdown 'https://drive.google.com/uc?id=1oBzZdtg2zNTPGhbRy6DQ_wrf5L5OAhNR' -O cifar.npz

with np.load('cifar.npz') as data:
    X_trn = data['train_data']
    y_trn = data['train_labels']
    X_tst = data['test_data']
    y_tst = data['test_labels']

# testing

In [2]:
C          = -1
PATCH_SIZE = 4
PATCH_NUM  = 1000000
STRIDE     = 4
K          = 64

In [3]:
extractor = FeaturesExtractor(PATCH_SIZE, PATCH_NUM, STRIDE, K)

extractor.fit(X_trn)

X_trn_features = extractor.extract(X_trn)
X_tst_features = extractor.extract(X_tst)

y_trn = keras.utils.to_categorical(y_trn)
y_tst = keras.utils.to_categorical(y_tst)

In [ ]:
M = X_trn_features.shape[1]

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(M,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_trn_features, y_trn, batch_size=512, epochs=20, verbose=1, validation_data=(X_tst_features, y_tst))

loss, accuracy = model.evaluate(X_tst_features, y_tst, verbose=0)
print('Accuracy:', accuracy)

Epoch 1/20
98/98 [==============================] - 4s 41ms/step - loss: 1.8948 - accuracy: 0.4105 - val_loss: 1.5213 - val_accuracy: 0.4681
Epoch 2/20
98/98 [==============================] - 4s 42ms/step - loss: 1.2270 - accuracy: 0.5807 - val_loss: 1.4843 - val_accuracy: 0.4963
Epoch 3/20
98/98 [==============================] - 4s 43ms/step - loss: 0.9467 - accuracy: 0.6764 - val_loss: 1.5434 - val_accuracy: 0.4944
Epoch 4/20
98/98 [==============================] - 4s 42ms/step - loss: 0.7035 - accuracy: 0.7638 - val_loss: 1.6342 - val_accuracy: 0.5091
Epoch 5/20
98/98 [==============================] - 4s 42ms/step - loss: 0.5033 - accuracy: 0.8319 - val_loss: 1.7785 - val_accuracy: 0.5058
Epoch 6/20
98/98 [==============================] - 4s 42ms/step - loss: 0.3532 - accuracy: 0.8846 - val_loss: 2.0278 - val_accuracy: 0.4985
Epoch 7/20
98/98 [==============================] - 4s 43ms/step - loss: 0.2629 - accuracy: 0.9146 - val_loss: 2.2195 - val_accuracy: 0.5043
Epoch 8/20
98